# Install Dependencies

In [68]:
!pip install -U pip beautifulsoup4

  Using cached pip-22.2.1-py3-none-any.whl (2.0 MB)
  Using cached beautifulsoup4-4.11.1-py3-none-any.whl (128 kB)


ERROR: To modify pip, please run the following command:
C:\Users\divanma\.conda\envs\windows-ml-station\python.exe -m pip install -U pip beautifulsoup4


# Import Dependencies

In [69]:
from bs4 import BeautifulSoup
import requests

# Config

In [70]:
asset_code = 'AAPL'
asset_summary_url = f'https://finviz.com/quote.ashx?t={asset_code}'

# Implementation

In [71]:
def get_html_page(url: str) -> object:
    response: requests.Response = requests.get(url=url, headers={
        'Cache-Control': 'no-cache',
        'User-Agent': 'PostmanRuntime/7.29.0',
        'Accept': '*/*',
        'Accept-Encoding': 'gzip, deflate, br',
        'Connection': 'keep-alive'
    })
    response_text: str = response.text

    return BeautifulSoup(response_text, 'html.parser')

In [72]:
asset_summary_html = get_html_page(url=asset_summary_url)

In [108]:
eps_ttm: float = float(asset_summary_html.find(text='EPS (ttm)').find_next(class_='snapshot-td2').text)
eps_five_years: float = float(asset_summary_html.find(text='EPS next 5Y').find_next(class_='snapshot-td2').text.replace('%', '')) / 100
pe_ratio: float = float(asset_summary_html.find(text='P/E').find_next(class_='snapshot-td2').text)
forward_pe_ratio: float = float(asset_summary_html.find(text='Forward P/E').find_next(class_='snapshot-td2').text)
avg_pe_ratio: float = (pe_ratio + forward_pe_ratio) / 2
current_price: float = float(asset_summary_html.find(text='Price').find_next(class_='snapshot-td2').text)

print(f'EPS: {eps_ttm}, EPS Next 5 Years: {eps_five_years}%, P/E Ratio: {avg_pe_ratio}, Current Price: $ {current_price}')

EPS: 6.05, EPS Next 5 Years: 0.0983%, P/E Ratio: 25.995, Current Price: $ 162.51


In [101]:
def determine_intrinsic_value(eps_ttm: float, # Total trailing annual earnings per share.
                              growth_rate: float, # Projected 5 year EPS.
                              pe_ratio: float, # Price per earnings growth rate. 2x growth_rate if unsure.
                              min_rate_of_return: float = 0.15, # Rate of return we want to make.
                              margin_of_safety: float = 0.3 # Margin to padd our valuation with in order to mitigate risk. 20-50% usually.
                              ) -> float:
    assert eps_ttm > 1
    assert growth_rate > 0 and growth_rate < 1
    assert pe_ratio > 1
    assert min_rate_of_return > 0 and min_rate_of_return < 1
    assert margin_of_safety > 0 and margin_of_safety < 1
    
    target_ten_year_eps: float = eps_ttm

    for year in range(2, 11):
        target_ten_year_eps *= (1 + eps_five_years)

    target_ten_year_share_price: float = target_ten_year_eps * avg_pe_ratio
    target_share_price: float = target_ten_year_share_price

    for year in range(2, 11):
        target_share_price /= (1 + growth_rate)

    return target_share_price / (1 + margin_of_safety)

In [109]:
intrinsic_value: float = determine_intrinsic_value(eps_ttm=eps_ttm,
                          growth_rate=eps_five_years,
                          pe_ratio=avg_pe_ratio,
                          margin_of_safety=0.3)
intrinsic_value = round(intrinsic_value, ndigits=2)

print(f'Intrinsic Value: $ {intrinsic_value} vs. Current Price: $ {current_price}')

Intrinsic Value: $ 120.98 vs. Current Price: $ 162.51
